Este notebook tem por objetivo testar a capacidade de um modelo de aprendizagem de máquina de acertar a seguinte pergunta:  
  - Dado esse bloco de texto, extraído de algum Diário Oficial do Distrito Federal, ele possui ou não atos de pessoal?
  
Para isso será treinado um modelo de *boosting*, o XGB, e a estratégia de validação adotada foi a [*Leave One Out*](https://scikit-learn.org/stable/modules/cross_validation.html).  
Como representação do texto será utilizado o clássico **tfidf** e não serão removidas *stop words*.

In [ ]:
# %load_ext autoreload
# %autoreload 2

import os
from itertools import chain

import pandas as pd

from sklearn.model_selection import LeaveOneOut
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
import xgboost as xgb
import matplotlib.pyplot as plt

import fitz
import helper

METRICS = [
    'f1_macro',
    'f1_micro',
    'f1_weighted',
    'accuracy',
    'balanced_accuracy'
]

In [ ]:
PATH_PREDICT = 'marked_pdf/predict/'
PATH_REGEX = 'marked_pdf/regex/'

os.makedirs(PATH_PREDICT, exist_ok=True)
os.makedirs(PATH_REGEX, exist_ok=True)
os.makedirs(PATH_PREDICT[:-1] + '_csv', exist_ok=True)
os.makedirs(PATH_REGEX[:-1] + '_csv', exist_ok=True)

Abaixo é realizado a extração dos *blocos* de cada arquivo **.pdf** e, em seguida, cada bloco é rotulado:
  - **True**, caso  haja ao menos um ato nesse bloco
  - **False** caso não seja detectado nenhum ato

Além disso, são salvos os seguintes metadados acerca desses blocos:
  - coordenadas (x0, y0, x1, y1)
  - número da página (todo bloco começa e termina numa só página)
Por fim, cada linha do *dataframe* possui um campo de texto chamado *text*.


In [ ]:
%%time
file_lis = ['pdf/'+i for i in os.listdir('pdf') if i.endswith('.pdf')]
file_df_dict = {}

cond = lambda x: len(x[4]) > 40 and '\n' in x[4]
for fname in file_lis:
    doc = fitz.open(fname)
    blocks = list(
        chain(
            *[ [(*i, pnum) for i in p.getTextBlocks() if cond(i)]
              for (pnum, p) in enumerate(doc)]
        )
    )
    pars = [i[4] for i in blocks]
    file_df_dict[fname] = pd.DataFrame({
        'text': pars,
        'pnum':[int(i[-1]) for i in blocks],
        'x0': [i[0] for i in blocks],
        'y0': [i[1] for i in blocks],
        'x1': [i[2] for i in blocks],
        'y1': [i[3] for i in blocks],
        'y': map(helper.has_act, pars)
    })

Dado o número reduzido de documentos a serem usados nos experimentos, foi possível adotar a estratégia **LeaveOneOut** como projeto do experimento.

In [ ]:
# %%time
index_map_path= dict(enumerate(file_df_dict))


pipes = {}
preds = {}
dfs = {}

all_keys = set(file_df_dict)

for train, test in LeaveOneOut().split(index_map_path):
    pipe = Pipeline([
        ('vectorizer', TfidfVectorizer()),
        ('model', xgb.XGBClassifier(
            objective='multi:softprob',
            random_state=42,
            num_class=2
        ))
    ])    
    
    df = pd.concat(map(lambda x: file_df_dict[x], [index_map_path[i] for i in train]))
    pipe.fit(df.text, df.y);
    
    out = index_map_path[test[0]]
    
    pipes[out] = pipe
    dfs[out] = file_df_dict[out].text
    preds[out] = pipe.predict(file_df_dict[out].text)
    df = file_df_dict[out]
    X,y = df.text, df.y
    disp = plot_confusion_matrix(pipe, X, y,
                            cmap=plt.cm.Blues,
                            normalize='true',
                            xticks_rotation='vertical',
                            )
    f1 = f1_score(y, preds[out])
    acc = accuracy_score(y, preds[out])
    print(out)
    print(f'\tf1: {f1:.2f}, acc: {acc:.2f}')
    disp.ax_.set_title(f'left out - {out}')

Os arquivos **.pdf** lidos pertencem à pasta **pdf**.  

A seguir, são salvos versões dos arquivos lidos porém com a diferença que essas versões são **anotadas** com retângulos nos blocos cuja presença de algum ato foi detectada.  
Os arquivos **.pdf** são salvos nos diretórios:
  - PATH_PREDICT
  - REGEX_PREDICT

e seus "análogos" (seu blocos) no formato **csv** encontram-se em:
  - PATH_PREDICT, com sufixo `csv`
  - REGEX_PREDICT, com sufixo `csv`
  
> Nota: optou-se pela anotação em PDFs apenas para facilitar a inspeção visual dos resultados obtidos. Do ponto de vista computacional, os arquivos **.csv** fazem mais sentido.

In [ ]:
%%time
def dump_predict(lis, dist_dir, c=(.23, .41, .88)):
    for fname in lis:    
        doc = fitz.open(fname)
        df = file_df_dict[fname]
        clf = pipes[fname]
        X, y = df['text'], df['y']
        predict = clf.predict(X)
        trues = df[predict == True]
        [doc[int(i.pnum)].drawRect(i[2:6], color=c, width=1)
            for i in trues.iloc];
        doc.save(dist_dir + fname.split('/')[-1][:-4] + '_predict.pdf');

def dump_regex(lis, dist_dir, c=(0, .5, .26)):
    for fname in lis:    
        doc = fitz.open(fname)
        df = file_df_dict[fname]
        clf = pipes[fname]
        trues = df[df.y == True]
        if not any(trues):
            print("skip", fname)
        [doc[int(i.pnum)].drawRect(i[2:6], color=c, width=1)
            for i in trues.iloc];
        doc.save(dist_dir + fname.split('/')[-1][:-4] + '_regex.pdf');

def dump_csv():
    def pdf2csv(s):
        return s.split('/')[-1][:-3]+'csv'

    for fname in file_lis:    
        df = file_df_dict[fname]
        prediction = preds[fname]
        dest = PATH_REGEX[:-1] + '_csv/' + pdf2csv(fname)
        print("FILE_NAME:", fname)
        print("DEST_PATH:", dest)
#         input()
        df.to_csv(dest, index=False)
        pd.concat([df.iloc[:, :-1], pd.Series(prediction, name='y')], axis=1).to_csv(
            PATH_PREDICT[:-1] + '_csv/' + pdf2csv(fname), index=False
        )
# dump_regex(file_lis, PATH_REGEX)
# print("Dumped regex")
# dump_predict(file_lis, PATH_PREDICT)
# print("Dumped predict")
# dump_csv()
# print("Dumped csv")

Conclusões preliminares:
  - XGB mostrou-se bastante eficaz em detectar blocos contendo atos
  - XGB encontrou blocos que não tinham sido apontados como possuindo atos de pessoal mas que de fato os possuíam
  - Promissora a ideia de detecção automatizada de blocos não capturados pelas expressões regulares

Limitações:
  - falta de dados anotados, o que permitiria comparação mais acurada entre ambas rotulações
  - outros classificadores não foram testados. Por exemplo, o SVM poderia diminuir imensamente o tempo e talvez produzir resultados semelhantes.